In [6]:
import re
import os
import io
import sys
import requests
import tiktoken
import pandas as pd
from tqdm import tqdm
from openai import OpenAI
from dotenv import load_dotenv
from colorama import Fore, Style, init
from googleapiclient.discovery import build

load_dotenv()
GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')
GOOGLE_CSE_ID = os.getenv('GOOGLE_CSE_ID')
google_fact_check_api_key = os.getenv('google_fact_check_api_key')
API_KEY = os.getenv('OPENAI_API_KEY')

client = OpenAI(api_key=API_KEY)

# Initialize colorama
init()

class FactChecker:
    def __init__(self):
        self.messages = []
        self.tools_used = False
        self.model = "gpt-3.5-turbo"
        self.available_functions = {
            "call_google": self.call_google,
        }

    def set_model(self, model_name):
        """更改模型"""
        self.model = model_name
    
    def call_google(self, query, **kwargs):
        try:
            service = build(serviceName="customsearch",
                          version="v1",
                          developerKey=GOOGLE_API_KEY,
                          static_discovery=False)
            res = service.cse().list(q=query, cx=GOOGLE_CSE_ID, **kwargs).execute()
            res_items = res["items"]
            res_snippets = [r['snippet'] for r in res_items]
            self.tools_used = True
            return str(res_snippets)
        except Exception as e:
            print(Fore.RED + f"Error: {str(e)}")
            print(Style.RESET_ALL)
            return f"Error in Google search: {str(e)}"

    def extract_sub_questions(self, article):
        """抽取子問題的prompt"""
        prompt = f"""
        Please analyze the following article and break it down into 2-3 key factual claims 
        that need to be verified. Format them as numbered questions:
        
        Article: {article}
        
        Please list the sub-questions in this format:
        1. [First question]
        2. [Second question]
        3. [Third question]
        """
        
        messages = [
            {"role": "system", "content": "You are an expert at breaking down articles into key verifiable claims."},
            {"role": "user", "content": prompt}
        ]
        
        response = client.chat.completions.create(
            model=self.model,
            messages=messages
        )
        
        return response.choices[0].message.content
    
    def verify_sub_question(self, sub_question, article):
        """驗證單個子問題"""
        self.messages = []
        self.tools_used = False
        sys_prompt = """
        You are a fact-checking specialist. Verify the specific claim by:
        1. Thinking about what you already know and what you need to verify
        2. Use tool to find the information:
            - call_google: For general information and recent events
        3. Analyzing the search results
        4. Providing a conclusion
    
        You must use this format:
        Thought: [Your reasoning]
        Action: [call_google]: [search query]
    
        After receiving an observation:
        Thought: [Analysis combining your knowledge and the new information]
        Action: [Another search if needed]
        or
        Answer: [Your final conclusion]
    
        Available actions:
        call_google: [search term]
        """

        user_prompt = f"Sub-question: {sub_question}\nArticle context: {article}\n\nPlease verify this claim using both your knowledge and external sources when needed."
    
        self.messages = [
            {"role": "system", "content": sys_prompt},
            {"role": "user", "content": user_prompt}
        ]
    
        return self._run_verification_loop()

    def _run_verification_loop(self):
        action_re = re.compile('^Action: (\w+): (.*)$')
        answer_re = re.compile("Answer: ")
        count = 0
        last_answer = None  # 記錄最後一個有效的 Answer
        
        while True:
            # 修改 messages 以適應 token 限制
            self.messages = self.trim_messages_to_fit(self.messages, max_tokens=4096)

            response = client.chat.completions.create(
                model=self.model,
                messages=self.messages
            )
            
            response_msg = response.choices[0].message.content
            self.messages.append({"role": "assistant", "content": response_msg})

            # 嘗試提取 Answer
            extracted_answer = self._extract_answer(response_msg)
            if extracted_answer:
                last_answer = extracted_answer  # 更新最後提取到的有效 Answer
            
            # 檢查工具是否被使用並有 Answer
            if extracted_answer and self.tools_used:
                print(Fore.YELLOW + response_msg)
                print(Style.RESET_ALL)
                return extracted_answer

            # 如果有 Answer 但工具未被使用
            elif extracted_answer:
                print(Fore.YELLOW + "Answer found but tool has not been used. Searching tools now.")
                print(Style.RESET_ALL)

            print(Fore.GREEN + response_msg)
            print(Style.RESET_ALL)
            
            # 處理回應中的 Action
            actions = [action_re.match(a) for a in response_msg.split("\n") if action_re.match(a)]
            if actions:
                action, action_input = actions[0].groups()
                try:
                    print(Fore.CYAN + f" -- running {action} {action_input}")
                    print(Style.RESET_ALL)
                    
                    observation = self.available_functions[action](action_input)
                    self.tools_used = True
                    
                    if "No claims found" in observation:
                        self.tools_used = False

                    print(Fore.BLUE + f"Observation: {observation}")
                    print(Style.RESET_ALL)
                    self.messages.append({"role": "user", "content": "Observation: " + observation})
                
                except Exception as e:
                    print(Fore.RED + f"Error: {e}")
                    print(Style.RESET_ALL)
                    continue
            
            # 沒有動作被檢測到
            else:
                count += 1
                if count == 3:
                    print(Fore.RED + "No action is detected. The answer is Unknown.")
                    print(Style.RESET_ALL)
                    
                    # 如果有記錄到有效的 Answer
                    if last_answer:
                        print(Fore.GREEN + f"Returning last extracted Answer: {last_answer}")
                        print(Style.RESET_ALL)
                        return last_answer
                    return "Ignore this question"
                
                print(Fore.RED + "No action is detected. Continue...")
                self.tools_used = False
                continue

    def trim_messages_to_fit(self, messages, max_tokens=16385):
        """刪除前面的tokens以適應 token 限制"""
        encoding = tiktoken.encoding_for_model(self.model)

        # 如果第一條是 system prompt，暫時移除
        system_message = None
        if messages and messages[0]["role"] == "system":
            system_message = messages.pop(0)

        while self.count_tokens(messages) > max_tokens:
            # 刪除最早的tokens，直到符合限制
            messages.pop(0)

        # 如果有 system prompt，重新插入到最前面
        if system_message:
            messages.insert(0, system_message)

        return messages
    
    def count_tokens(self, messages, model=None):
        """計算 token 數"""
        model = self.model 
        encoding = tiktoken.encoding_for_model(model)
        total_tokens = 0
        for message in messages:
            for key, value in message.items():
                total_tokens += len(encoding.encode(value))
        return total_tokens
            
    def _extract_answer(self, response_msg):
        """從回應中提取答案"""
        lines = response_msg.split('\n')
        for line in lines:
            if 'Answer:' in line:
                try:
                    # 取 'Answer:' 後的內容
                    answer = line.split('Answer:')[1].strip()
                    print(f"Extracted Answer: {answer}")
                    return answer
                except IndexError as e:
                    print(f"Error extracting answer: {e}")
                    continue
        return None
    

    def _execute_action(self, action, action_input):
        """執行搜尋動作"""
        if action in self.available_functions:
            print(Fore.CYAN + f" -- running {action} {action_input}")
            print(Style.RESET_ALL)
            result = self.available_functions[action](action_input)
            print(Fore.BLUE + f"Observation: {result}")
            print(Style.RESET_ALL)
            return result
        else:
            print(Fore.RED + f"Unknown action: {action}")
            print(Style.RESET_ALL)
            return f"Unknown action: {action}"
                
    def combine_results(self, results, article):
        """綜合所有子問題的結果"""
        prompt = f"""
        Original article:
        {article}

        Based on the verification results of all sub-questions:
        {results}
    
        Please provide a final conclusion about whether the article contains misleading information.
        First provide your detailed reasoning, then end with either:
        Final Answer: 1 (if misleading)
        Final Answer: 0 (if accurate)
        """
    
        messages = [
            {"role": "system", "content": "You are a fact-checking expert providing final conclusions based on detailed analysis of multiple claims."},
            {"role": "user", "content": prompt}
        ]

        # 印出組好的 Prompt
        print("\n=== Debug: Prompt for LLM ===")
        print(prompt)
        print("=============================")
    
        response = client.chat.completions.create(
            model=self.model,
            messages=messages
        )

        return response.choices[0].message.content

    def verify_article(self, article):
        """主要驗證流程"""
        print(Fore.CYAN + "Extracting sub-questions..." + Style.RESET_ALL)
        sub_questions = self.extract_sub_questions(article)
        print(Fore.YELLOW + "Sub-questions extracted:\n" + sub_questions + Style.RESET_ALL)
        
        results = []
        for q in sub_questions.split("\n"):
            if q.strip() and q[0].isdigit():
                print(Fore.CYAN + f"\nVerifying: {q}" + Style.RESET_ALL)
                result = self.verify_sub_question(q, article)
                results.append(f"Question: {q}\nResult: {result}")
        
        print(Fore.CYAN + "\nCombining results..." + Style.RESET_ALL)
        final_conclusion = self.combine_results(results, article)
        print(Fore.YELLOW + "\nFinal conclusion:\n" + final_conclusion + Style.RESET_ALL)
        
        return final_conclusion
    
    def _clean_ansi_codes(self, text):
        """移除 ANSI 顏色代碼"""
        ansi_escape = re.compile(r'\x1B(?:[@-Z\\-_]|\[[0-?]*[ -/]*[@-~])')
        return ansi_escape.sub('', text)
    
    def process_article_batch(self, input_file, output_file, num_files=None):
        """處理整個Excel中的文章"""
        log_file = "fact_checking_process.log"
        
        try:
            with open(log_file, 'w', encoding='utf-8') as f:
                try:
                    df = pd.read_csv(input_file, encoding='utf-8-sig')
                except UnicodeDecodeError:
                    df = pd.read_csv(input_file, encoding='latin1')
        
                if num_files is not None:
                    df_to_process = df.head(num_files)
                else:
                    df_to_process = df
            
                df['fact_check_response'] = ''
                df['Simplified Answer'] = ''

                df.to_csv(output_file, index=False, encoding='utf-8-sig')
                print("開始處理文章...")
        
                for idx, row in tqdm(df_to_process.iterrows(), total=len(df_to_process), desc="Processing articles"):
                    
                    # article = row['synthetic_misinformation']    # LLMFake 
                    article = row['text']   # mmsoc_gossipcop_text_labels-100 

                    output_capture = io.StringIO()
                    original_stdout = sys.stdout
                    sys.stdout = output_capture

                    try:
                        result = self.verify_article(article)
                    finally:
                        sys.stdout = original_stdout

                    detailed_output = output_capture.getvalue()
                    cleaned_output = self._clean_ansi_codes(detailed_output)    
                    simplified_answer = self._extract_simplified_answer(result)

                    f.write(detailed_output)
                    f.flush()

                    current_df = pd.read_csv(output_file, encoding='utf-8-sig')

                    current_df.at[idx, 'fact_check_response'] = cleaned_output
                    current_df.at[idx, 'Simplified Answer'] = simplified_answer
            
                    current_df.to_csv(output_file, index=False, encoding='utf-8-sig')

                print(f"\nAll articles processed. Results saved to {output_file}", file=original_stdout)
        
        except Exception as e:
            print(f"Error processing batch: {str(e)}", file=original_stdout)
            raise e
  
    def _extract_simplified_answer(self, response):
        """從回應中提取簡化的答案 (1 或 0)"""
        try:
            match = re.search(r"Final Answer:\s*(\d)", response)
            if match:
                return int(match.group(1))
        
            digits = re.findall(r'[01]', response)
            if digits:
                return int(digits[-1])
            else:
                return "Unknown"
        except Exception:
            return "Error"

    

### Analysis datasets

In [7]:
def main():
    input_file = "mmsoc_gossipcop_text_labels-100.csv"  # if LLMFake file，revise above to synthetic_misinformation
    output_file = "react_child_task_google_gossipcop-100.csv"
    num_files_to_process = 100
    
    checker = FactChecker()
    checker.set_model("gpt-3.5-turbo")  # 設定使用的模型
    
    try:
        checker.process_article_batch(input_file, output_file, num_files=num_files_to_process)
        
    except Exception as e:
        print(Fore.RED + f"Error in main execution: {str(e)}" + Style.RESET_ALL)
        sys.exit(1)

if __name__ == "__main__":
    main()

開始處理文章...


Processing articles:   0%|          | 0/100 [00:00<?, ?it/s]

/tmp/ipykernel_2197017/1504838315.py:353: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Extracting sub-questions...
Sub-questions extracted:
1. Did Cher perform at the 2017 Billboard Music Awards with two performances of her greatest hits and receive the Billboard Icon Award?
2. Has Cher sold more than 100 million albums worldwide and achieved a No. 1 single in each decade from the 1960s to the 2010s, making her the first artist to accomplish this according to Billboard charts?
3. Did Gwen Stefani present Cher with the Billboard Icon Award, describing her as a trailblazer, fashion trendsetter, role model, cultural influencer, activist, and humanitarian during the award ceremony?

Verifying: 1. Did Cher perform at the 2017 Billboard Music Awards with two performances of her greatest hits and receive the Billboard Icon Award?
Thought: This claim can be fact-checked by looking for information related to Che


All articles processed. Results saved to react_child_task_google_gossipcop-100.csv


### Analysis single article

In [3]:
checker = FactChecker()
checker.set_model("gpt-3.5-turbo") # 設定使用的模型
article = """Together, Meghan Markle and Prince Harry will be worth about $30 million — and none of that money belongs to the crown Prince Harry's net worth is 8-figures. Chris Jackson - WPA Pool/Getty Images

Megan Markle will be marrying into class, status, and major wealth when she ties the royal knot with Prince Harry on May 19.

Before she moved across the pond, Markle was earning close to half a million dollars a year starring in the USA Network drama "Suits," plus a five-figure income from endorsement deals and sponsorships. Her total net worth is estimated to be $5 million.

That's a sizable contribution to bring into any marriage, but when she marries Prince Harry, their combined net worth will be more than five times that. Prince Harry's net worth is at least $25 million (£18 million) and as much as $40 million (£28.7 million), according to estimates from Wealth-X.

Here's where all that money comes from.

Along with Prince William and Kate Middleton, Harry receives an annual seven-figure allowance from his father and heir to the throne, Prince Charles, which is used to cover expenses like travel and wardrobe. Between April 2016 and March 2017, the Duke and Duchess of Cambridge and Harry split nearly $5 million (£3.5 million) in allowance from their father, according to an annual review released by Clarence House.

Prince Charles derives that money, in part, from a private estate called the Duchy of Cornwall, which has funded the royal family's public and private lives for nearly 700 years. The estate provided a total of $28.8 million (£20.7 million), including Harry and William's allowance, to Prince Charles and his wife Camilla between 2016 and 2017.

Chris Jackson/Getty

Since age 21, Harry and William have also been receiving a $450,000 a year investment profit from Princess Diana's estate, which they pay taxes on to the UK government. They received a sizeable inheritance from their late mother estimated to be around $10 million each, which each prince gained access to on his 30th birthday. Plus, during his 10 years in the British Royal Air Force, Harry earned an annual income of $50,000.

Despite a massive fortune, Markle and Harry reportedly will not sign a prenuptial agreement before their wedding, reported Business Insider's Shana Lebowtiz.

A prenup is a smart choice for most couples who have sizable assets they want to protect in the event of a divorce, but for Harry — and William, who chose to forgo a prenup with Middleton — it doesn't make much sense. The princes likely don't have much property of their own, as the most valuable national treasures like the Crown Jewels or the Tower of London are part of the royal collection held by the Queen.

Still, the merging of Harry and Markle's finances could "cause tax headaches" and create some "mundane hurdles" for the royal family, as The Washington Post first reported in November.

Markle is a citizen of the US and is purportedly living in the UK on a family visa, according to the BBC. If she eventually becomes a dual US-UK citizen, Markle will have to continue filing her taxes each year with the IRS. If she has more than $300,000 in assets at any point during the year, she will have to file a specific form that details foreign assets, which could include foreign trusts, subjecting the royal family "to outside scrutiny," according to the Post. But ultimately there are a lot of factors that come into play.

Eddie Mulholland/AP

"The key for Meghan and her advisors would be to figure out what type of income she will be getting," Avani Ramnani, director of financial planning and wealth management at Francis Financial, told Business Insider. "Will this income be from the investments of a trust, or 'wages' for any work that she does, or any other type of income? Sometimes, getting one form of income is more advantageous than another."

Kensington Palace will cover the cost of the wedding — aside from Markle's dress — which is expected to cost the royal family upwards of $45 million (£32 million), most of which is allotted for security.

That won't be a huge cost burden for Queen Elizabeth, who is of course the wealthiest member of the royal family with an estimated net worth of $530 million as of 2016, according to Forbes.

More on the royal wedding:"""
result = checker.verify_article(article)

Extracting sub-questions...


Sub-questions extracted:
1. What is the estimated combined net worth of Meghan Markle and Prince Harry when they get married?
2. Where does Prince Harry's estimated net worth of $25 million to $40 million come from?
3. What potential financial implications could arise for the royal family due to Meghan Markle's U.S. citizenship and financial assets?

Verifying: 1. What is the estimated combined net worth of Meghan Markle and Prince Harry when they get married?
Sure, I'll verify the claim regarding the estimated combined net worth of Meghan Markle and Prince Harry when they get married. Let's gather all the relevant information. 

Thought: Meghan Markle's estimated net worth is $5 million, while Prince Harry's net worth is estimated to be between $25 million and $40 million. 
Action: call_google: combined net worth of Meghan Markle and Prince Harry when they get married

 -- running call_google combined net worth of Meghan Markle and Prince Harry when they get married

Observation: ['..

### Caculate Accuracy

In [ ]:
import pandas as pd

file_name = "react_child_task_google_gossipcop-100"

data = pd.read_csv(f'{file_name}.csv')
data['Simplified Answer'] = data['Simplified Answer'].astype(str)

# 計算總體準確率（僅考慮 0 和 1）
valid_predictions = data[data['Simplified Answer'].isin(['0', '1'])]
overall_accuracy = (valid_predictions['label'] == valid_predictions['Simplified Answer'].astype(int)).mean()

# 計算 label=1（假新聞）的準確率
label_1_data = valid_predictions[valid_predictions['label'] == 1]
accuracy_label_1 = (label_1_data['label'] == label_1_data['Simplified Answer'].astype(int)).mean()
mistakes_label_1 = len(label_1_data) - (label_1_data['label'] == label_1_data['Simplified Answer'].astype(int)).sum()

# 計算 label=0（真新聞）的準確率
label_0_data = valid_predictions[valid_predictions['label'] == 0]
accuracy_label_0 = (label_0_data['label'] == label_0_data['Simplified Answer'].astype(int)).mean()
mistakes_label_0 = len(label_0_data) - (label_0_data['label'] == label_0_data['Simplified Answer'].astype(int)).sum()

# 統計 "Uncertain" 和 "error" 的情況
uncertain_count = len(data[data['Simplified Answer'] == 'Unknown'])
error_count = len(data[data['Simplified Answer'] == 'ERROR'])

print(f"檔案名稱: {file_name}.csv")
print(f"總體準確率: {overall_accuracy:.2f}")
print(f"label=1（假新聞）的準確率: {accuracy_label_1:.2f}, 錯誤數量: {mistakes_label_1}")
print(f"label=0（真新聞）的準確率: {accuracy_label_0:.2f}, 錯誤數量: {mistakes_label_0}")
print(f"Uncertain 的數量: {uncertain_count}")
print(f"ERROR 的數量: {error_count}")
